In [2]:
import os
import pandas as pd
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-10_2.12:3.3.0,org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.0,org.mongodb.spark:mongo-spark-connector_2.12:10.1.1 pyspark-shell'
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json, col, expr
from pyspark.sql.types import StructType, StringType, TimestampType
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.types import (
    StructType, StringType, IntegerType, DoubleType, TimestampType
)
from pyspark.sql.functions import (
    col, expr, from_json
)

spark = SparkSession.builder.appName("CameraJoin").master("local[*]").getOrCreate()

# 1. Define your JSON schema
schema = StructType() \
                        .add("batch_id", IntegerType()) \
                        .add("event_id", StringType()) \
                        .add("car_plate", StringType()) \
                        .add("camera_id", IntegerType()) \
                        .add("timestamp", TimestampType()) \
                        .add("speed_reading", DoubleType()) \
                        .add("producer", StringType()) \
                        .add("sent_at", StringType())
def read_camera(topic):
    return (
        spark.readStream
             .format("kafka")
             .option("kafka.bootstrap.servers", "172.22.32.1:9092")
             .option("subscribe", topic)
             .option("startingOffsets", "earliest")    # ← add this
             .load()
        # parse the JSON payload
             .select(from_json(col("value").cast("string"), schema).alias("j"))
             .select("j.*")
        # use the event timestamp for watermarks
             .withWatermark("timestamp", "10 hours")
    )

# 2. Read each stream with the same watermark
streamA = read_camera("camera_event_a")
streamB = read_camera("camera_event_b")
streamC = read_camera("camera_event_c")

# 3. Join A and B on car_plate within a ±2-minute window
joinAB = (
    streamA.alias("A").join(
        streamB.alias("B"),
        expr("""
          A.car_plate = B.car_plate AND
          B.timestamp BETWEEN A.timestamp AND A.timestamp + INTERVAL 10 HOURS
        """),
        how="inner"
    )
)


In [3]:
# 5. Write out your tri-stream joins
query = (
    joinAB
      .select("A.*", "B.*")
      .writeStream
      .format("console")
      .outputMode("append")
      .option("checkpointLocation", "./stream_checkpoints")
      .start()
)

query.awaitTermination()

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/opt/conda/lib/python3.8/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/opt/conda/lib/python3.8/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/opt/conda/lib/python3.8/socket.py", line 669, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 